In [ ]:
from download_day import download_day
from load_saved_fd4 import load_saved_fd4
from scrape_aircraft_list import scrape_aircraft_list
from get_perf_model_typecodes import get_perf_model_typecodes 
from match_icao_model import match_icao_model
import pandas as pd
from time import sleep

In [ ]:
# User Inputs:
start_time_str       = '2024-01-01T00:00:00Z'
stop_time_str        = '2024-01-10T00:00:01Z'
query_limit          = 30e4
send_notification    = True
make_plot            = False
output_dir           = "/scratch/omg28/Data/no_track2024"

In [ ]:
# Download data from OpenSky history database
download_day(start_time_str, stop_time_str, query_limit, send_notification, make_plot, output_dir)

In [ ]:
loaded_day = load_saved_fd4(stop_time_str, output_dir, query_limit)

In [ ]:
available_performance_typecodes = get_perf_model_typecodes()
aircraft_performance_database_1 = scrape_aircraft_list(available_performance_typecodes['typecode'][0:100])

aircraft_performance_database_2 = scrape_aircraft_list(available_performance_typecodes['typecode'][100:200])
sleep(5)  # Sleep to avoid overwhelming the server
aircraft_performance_database_3 = scrape_aircraft_list(available_performance_typecodes['typecode'][200:300])
sleep(5)
aircraft_performance_database_4 = scrape_aircraft_list(available_performance_typecodes['typecode'][300:len(available_performance_typecodes)])


In [8]:
print(available_performance_typecodes['typecode'])

Series([], Name: typecode, dtype: object)
